In [1]:
%matplotlib notebook

In [2]:
import firedrake as fd
from firedrake.meshadapt import RiemannianMetric, adapt
from firedrake_adjoint import pyadjoint
from firedrake.adjoint import get_solve_blocks
from pyroteus.metric import *
from pyroteus.recovery import *
from thetis.log import print_output, INFO

In [3]:
from turbine import forward_run

In [4]:
from opt_adapt.opt import OptimisationProgress, minimise

In [5]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
from time import perf_counter

In [6]:
rerun = True

### Step 1: Write functions for initialisation

In [7]:
base_tape = pyadjoint.Tape()
refined_tape = pyadjoint.Tape()
pyadjoint.set_working_tape(base_tape)

In [8]:
tm = fd.TransferManager()

def initial_mesh(*args, **kwargs):
    n = 2
    return fd.RectangleMesh(n*120, n*50, 1200, 500)

In [9]:
def initial_control(mesh):
    R = fd.FunctionSpace(mesh, "R", 0)
    return fd.Function(R).assign(250.0)

### Step 2. Write functions for solving the forward problem and adapting the mesh

In [10]:
def get_state(adjoint=False, tape=base_tape):
    """
    Extract the current state from the tape (velocity and
    elevation).
    
    :kwarg adjoint: If ``True``, return the corresponding
        adjoint state variables.
    """
    solve_block = get_solve_blocks()[0]
    return solve_block.adj_sol if adjoint else solve_block._outputs[0].saved_output

In [11]:
def hessian(mesh, **kwargs):
    """
    Recover the Hessian of each component of the state and
    intersect them.
    
    :kwarg adjoint: If ``True``, recover Hessians from the
        adjoint state, rather than the forward one.
    """
    uv, eta = get_state(**kwargs).split()
    V = fd.FunctionSpace(mesh, uv.ufl_element().family(), uv.ufl_element().degree())
    u = fd.interpolate(uv[0], V)
    v = fd.interpolate(uv[1], V)
    
    def hessian_component(f):
        """
        Recover the Hessian of a single component and
        scale it so that all of the components are of
        consistent metric complexity.
        """
        return space_normalise(hessian_metric(recover_hessian(f)), 1000.0, "inf")
        
    metric = metric_intersection(hessian_component(u), hessian_component(v), hessian_component(eta))
    print_output("Hessian intersection complete.")
    return metric

In [12]:
def adapt_hessian_based(mesh, target=1000.0, norm_order=1.0, **kwargs):
    """
    Adapt the mesh w.r.t. the intersection of the Hessians of
    each component of velocity and pressure.
    
    :kwarg target: Desired metric complexity (continuous
        analogue of mesh vertex count).
    :kwarg norm_order: Normalisation order :math:`p` for the
        :math:`L^p` normalisation routine.
    """
    metric = space_normalise(hessian(mesh), target, norm_order)
    enforce_element_constraints(metric, 1.0e-05, 500.0, 1000.0)
    print_output("Metric construction complete.")
    newmesh = adapt(mesh, RiemannianMetric(mesh).assign(metric))
    print_output("Mesh adaptation complete.")
    return newmesh

### 3. Explore the parameter space

In [15]:
if rerun or not os.path.exists("controls_uniform.npy") or not os.path.exists("qois_uniform.npy"):
    ctrls = []
    qois = []
    for c in np.linspace(50, 450, 101):
        with pyadjoint.stop_annotating():
            mesh = initial_mesh()
            R = fd.FunctionSpace(mesh, "R", 0)
            J = forward_run(mesh, fd.Function(R).assign(c))[0]
            print(f"y-coordinate: {c:.2f}, power output = {-J:9.4e}")
            ctrls.append(c)
            qois.append(J)
            np.save("controls_uniform", ctrls)
            np.save("qois_uniform", qois)

y-coordinate: 50.00, power output = 3.5336e+05
y-coordinate: 54.00, power output = 3.5444e+05
y-coordinate: 58.00, power output = 3.5522e+05
y-coordinate: 62.00, power output = 3.5582e+05
y-coordinate: 66.00, power output = 3.5626e+05
y-coordinate: 70.00, power output = 3.5653e+05
y-coordinate: 74.00, power output = 3.5726e+05
y-coordinate: 78.00, power output = 3.5770e+05
y-coordinate: 82.00, power output = 3.5797e+05
y-coordinate: 86.00, power output = 3.5810e+05
y-coordinate: 90.00, power output = 3.5806e+05
y-coordinate: 94.00, power output = 3.5847e+05
y-coordinate: 98.00, power output = 3.5860e+05
y-coordinate: 102.00, power output = 3.5857e+05
y-coordinate: 106.00, power output = 3.5838e+05
y-coordinate: 110.00, power output = 3.5804e+05
y-coordinate: 114.00, power output = 3.5814e+05
y-coordinate: 118.00, power output = 3.5796e+05
y-coordinate: 122.00, power output = 3.5762e+05
y-coordinate: 126.00, power output = 3.5712e+05
y-coordinate: 130.00, power output = 3.5646e+05
y-coo

KeyboardInterrupt: 

### 4. Run the hybrid PDE-constrained optimisation / mesh adaptation routine

In [ ]:
if rerun or not (os.path.exists("m_progress_uniform.npy") and os.path.exists("J_progress_uniform.npy")):
    cpu_timestamp = perf_counter()
    op = OptimisationProgress()
    g_opt = minimise(forward_run, initial_mesh(), initial_control, options={'disp': 1, 'lr': 0.01}, op=op)
    print(f"Uniform optimisation completed in {perf_counter()-cpu_timestamp:.2f}s")
    np.save("m_progress_uniform", np.array([m.dat.data[0] for m in op.m_progress]).flatten())
    np.save("J_progress_uniform", op.J_progress)
    np.save("dJdm_progress_uniform", np.array([dj.dat.data[0] for dj in op.dJdm_progress]).flatten())

In [ ]:
# logger = logging.getLogger("thetis_output")
# logger.setLevel(INFO)

In [ ]:
if rerun or not (os.path.exists("m_progress_hessian.npy") and os.path.exists("J_progress_hessian.npy")):
    cpu_timestamp = perf_counter()
    op = OptimisationProgress()
    options = {
        'disp': 2,
        'lr': 0.01,
        'target_base': 1000,
        'target_inc': 200,
        'target_max': 4000,
    }
    g_opt = minimise(forward_run, initial_mesh(), initial_control, adapt_fn=adapt_hessian_based, options=options, op=op)
    print(f"Uniform optimisation completed in {perf_counter()-cpu_timestamp:.2f}s")
    np.save("m_progress_hessian", np.array([m.dat.data[0] for m in op.m_progress]).flatten())
    np.save("J_progress_hessian", op.J_progress)
    np.save("dJdm_progress_hessian", np.array([dj.dat.data[0] for dj in op.dJdm_progress]).flatten())

In [ ]:
def plot_progress(axes, method, gradients=False, **kwargs):
    """
    Plot the progress of an optimisation run, in terms
    of control value vs. objective functional value.
    
    :arg axes: the figure axes to plot on
    :arg method: the optimisation method
    :kwarg gradients: should gradients also be plotted?
    :kwargs: to be passed to matplotlib's plotting functions
    """
    m = np.load(f"m_progress_{method}.npy")
    J = -np.load(f"J_progress_{method}.npy")/1000
    print(f"Converged offset ({method}): {abs(250 - m[-1]):.2f}m offset => {J[-1]:.2f} kW")
    axes.plot(m, J, 'x', **kwargs)
    kwargs.pop("label")
    axes.arrow(x=m[-1], y=J[-1]+20, dx=0, dy=-10, width=1.5, **kwargs)
    if gradients:
        dJdm = -np.load(f"dJdm_progress_{method}.npy")/1000
        dc = 5.0
        for c, f, g in zip(m, J, dJdm):
            x = np.array([c - dc, c + dc])
            axes.plot(x, f + g*(x - c), '-', **kwargs)

In [ ]:
fig, axes = plt.subplots()
axes.plot(np.load("controls_uniform.npy"), np.abs(np.load("qois_uniform.npy"))/1000, color='C0')
plot_progress(axes, "uniform", label="Fixed mesh", color='C1', gradients=False)
plot_progress(axes, "hessian", label="Hessian-based", color='C2', gradients=False)

axes.set_xlabel('$y$-coordinate of second turbine')
axes.set_ylabel('Power output ($\mathrm{kW}$)')
axes.set_xlim([80, 260])
axes.legend()
axes.grid(True, which='both')
plt.tight_layout()

### To do:

* Support more advanced optimisation routines.
* Support constrained optimisation methods - here we would really like $y_2\in[\epsilon, 250]$.